# 🔢 Multiclass, Multioutput and Calibration

<img src="https://i.imgur.com/KwAfLy7.png"/>

### Multiclass Classification

- Many models support this inherently (e.g., Bayes classifier)
- Many as well don't (e.g., Perceptron)

#### 🤔 Problem Statement

How do we make a multiclass classifier out of a binary classifier?

|                        | One-vs-Rest                           | One-vs-One                        |
|------------------------|---------------------------------------|-----------------------------------|
| **Idea**               | Trains one classifier per class, treating it as positive and the rest as negative. | Trains a binary classifier for each pair of classes, distinguishing between them. |
| **Training**           | Requires training n binary classifiers where n is the number of classes. Each classifier learns to distinguish one class from the rest. | Requires training (n * (n - 1)) / 2 binary classifiers, where n is the number of classes, each trained on data from only two classes. |
| **Inference**          | During inference, all classifiers are applied, and the class where the corresponding classifier reported highest score or probability is chosen  | During inference, a voting scheme (e.g., majority voting) is used among all pairwise classifiers to determine the final class label. |




Scikit-learn supports both:

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.linear_model import Perceptron

x_data, y_data = load_iris(return_X_y=True)
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.33, shuffle=True, random_state=0)
clf = OneVsOneClassifier(Perceptron(max_iter=1000, eta0=0.1, random_state=0))
clf.fit(x_train, y_train)
clf.predict(x_val)

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0, 2, 2, 0, 2, 2, 1, 0, 2, 1, 1, 2, 0, 2, 0,
       0, 1, 2, 2, 2, 2])

### Multioutput Regression or Classification

Scikit-learn trains a model for each target columns.

In [3]:
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
from sklearn.linear_model import Perceptron

x_data, y_data = make_multilabel_classification(n_classes=3, random_state=0)
print(x_data.shape, y_data.shape)

(100, 20) (100, 3)


In [4]:
clf = MultiOutputClassifier(Perceptron())
clf.fit(x_data, y_data)
clf.predict(x_data)[0]          # three predictions for the first data point

array([0, 1, 0])

### Probability Calibration

We need it for two reasons:
- Some classifiers output scores but we are interested in probabilities (and standard normalization can be inaccurate)

- Some classifiers tend to output biased probabilities which you some examples for [here](https://scikit-learn.org/stable/modules/calibration.html#calibration-curves)

- For instance, Naive Bayes tends to push probabilities to 0 or 1 depending on the degree of violation of its assumption and the number of variables.

Calibrating a classifier consists of fitting a regressor (called a calibrator) that maps the output of the classifier $f_i$ (as given by `decision_function` or `predict_proba`) to a calibrated probability in $[0, 1]$


$$p(y_{i}=1|f_{i})=\frac{1}{1+exp(Af_{i}+B)}.$$

The regressor simply learns $A$ and $B$ given a dataset of the model outputs $f_i$ and the true labels $y_i$.

In [72]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV

x_data, y_data = make_classification(n_samples=1000, n_features=30, n_redundant=15,  n_repeated=10, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.33, shuffle=True, random_state=0)

base_clf = GaussianNB()
calibrated_clf = CalibratedClassifierCV(base_clf, method="sigmoid", cv=3)

Before Calibration

In [73]:
base_clf.fit(x_train, y_train)
base_clf.score(x_val, y_val)

0.806060606060606

After Calibration

In [74]:
calibrated_clf.fit(x_train, y_train)
calibrated_clf.score(x_val, y_val)

0.8121212121212121

**Finally our machine learning engineer journey is done**

<div align="center">
<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExNmJkcXQ1eDRtbW1ncmh6amppc2dtcGZ2aWFzdTllNHNnMHR6N3E5eSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/lD76yTC5zxZPG/giphy.gif">
</div>